In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np 
import pandas as pd 
import glob
import os

In [6]:
test_chat = glob.glob('/content/drive/MyDrive/projet_uqac/bdd/test_set/test_set/cats/' + '*jpg' )
test_chien = glob.glob('/content/drive/MyDrive/projet_uqac/bdd/test_set/test_set/dogs/' + '*jpg' )
train_chat = glob.glob('/content/drive/MyDrive/projet_uqac/bdd/training_set/training_set/cats/' + '*jpg' ) 
train_chien = glob.glob('/content/drive/MyDrive/projet_uqac/bdd/training_set/training_set/dogs/' + '*jpg' )

x_test = test_chat + test_chien
y_test = ['0' for i in range(len(test_chat))] + ['1' for i in range(len(test_chien))]

x_train = train_chat + train_chien
y_train = ['0' for i in range(len(train_chat))] + ['1' for i in range(len(train_chien))]

In [7]:
print(len(test_chat))
print(len(test_chien))
print(len(x_test))
print(len(y_test))
print('\n')
print(len(train_chat))
print(len(train_chien))
print(len(x_train))
print(len(y_train))

1011
1012
2023
2023


4000
4005
8005
8005


In [8]:
train_df = pd.DataFrame({'filename':x_train, 'label':y_train})
test_df = pd.DataFrame({'filename':x_test, 'label':y_test})

In [9]:
import tensorflow as tf
# Transform Image
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    #width_shift_range = 0.2,
    #height_shift_range = 0.2,
    validation_split = 0.2
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255
)

In [10]:
train_img = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'filename',
    y_col = 'label',
    target_size = (64, 64),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed =42,
    subset = 'training'
)
val_img = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'filename',
    y_col = 'label',
    target_size = (64, 64),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
)

test_img = test_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'filename',
    y_col = 'label',
    target_size = (64, 64),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = False
)

Found 6404 validated image filenames belonging to 2 classes.
Found 1601 validated image filenames belonging to 2 classes.
Found 2023 validated image filenames belonging to 2 classes.


In [11]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model

model = tf.keras.models.Sequential()

#STEP 1: ADDING CONVOLUTIONAL LAYER
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

model.add(tf.keras.layers.Conv2D(filters = 64,kernel_size = 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

model.add(tf.keras.layers.Conv2D(filters = 128,kernel_size = 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(units=256, activation='relu'))

model.add(tf.keras.layers.Dense(units=2, activation='softmax'))

In [12]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [14]:
model.fit(
    train_img,
    validation_data = val_img,
    epochs = 15,
    steps_per_epoch = 150
)

Epoch 1/15
150/150 [==============================] - 278s 2s/step - loss: 0.6010 - accuracy: 0.6708 - val_loss: 0.7214 - val_accuracy: 0.5440
Epoch 3/15
150/150 [==============================] - 87s 579ms/step - loss: 0.5387 - accuracy: 0.7250 - val_loss: 0.6655 - val_accuracy: 0.6671
Epoch 4/15
150/150 [==============================] - 42s 278ms/step - loss: 0.4842 - accuracy: 0.7657 - val_loss: 0.7266 - val_accuracy: 0.6171
Epoch 5/15
150/150 [==============================] - 32s 215ms/step - loss: 0.4468 - accuracy: 0.7940 - val_loss: 0.6459 - val_accuracy: 0.6852
Epoch 6/15
150/150 [==============================] - 29s 196ms/step - loss: 0.4254 - accuracy: 0.8039 - val_loss: 0.3851 - val_accuracy: 0.8457
Epoch 7/15
150/150 [==============================] - 29s 192ms/step - loss: 0.4035 - accuracy: 0.8233 - val_loss: 0.5264 - val_accuracy: 0.7676
Epoch 8/15
150/150 [==============================] - 29s 191ms/step - loss: 0.3743 - accuracy: 0.8330 - val_loss: 0.5472 - val_accu

In [15]:
model.save("1_model.h5")